# MLOps Sentiment Analysis - Sistema di Monitoraggio Completo

Questo notebook implementa il sistema completo di monitoraggio e retraining per il modello di sentiment analysis. Include:
- Download del dataset
- Valutazione del modello
- Logging delle predizioni
- Tracking delle metriche
- Rilevazione del drift
- Trigger di retraining
- Visualizzazioni dei risultati

In [1]:
from datasets import load_dataset

# Correct dataset path
dt = load_dataset("tweet_eval")

# Print sample counts
print(f"Train samples: {len(dt['train'])}")
print(f"Test samples: {len(dt['test'])}")

/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Repo card metadata block was not found. Setting CardData to empty.
Generating train split: 240540 examples [00:00, 1650111.36 examples/s]
Generating validation split: 21894 examples [00:00, 1657343.56 examples/s]
Generating test split: 139136 examples [00:00, 1802058.68 examples/s]

Train samples: 240540
Test samples: 139136


In [1]:
# Esempio di inferenza di base
from src.sentiment_model import analyze_sentiment

text = "Oggi è una bella giornata!"
result = analyze_sentiment(text)
print(f"Testo: {text}")
print(f"Risultato: {result}")

/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 201/201 [00:00<00:00, 548.22it/s, Materializing param=roberta.encoder.layer.11.output.dense.weight]              
RobertaForSequenceClassification LOAD REPORT from: /workspaces/mlops-ex/twitter-roberta-base-sentiment
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Testo: Oggi è una bella giornata!
Risultato: {'Negativo': 0.0330863781273365, 'Neutro': 0.6953458189964294, 'Positivo': 0.27156782150268555}


## Valutazione del Modello su Test Set

Valutiamo il modello su un campione del test set calcolando accuracy e F1 score.

In [2]:
from datasets import load_dataset, Dataset
import os

# Try loading tweet_eval default builder
dataset = load_dataset("tweet_eval")
# Use the test split if available
test_dataset = dataset['test']

# If the loaded test split has no label fields, attempt to load local files for the 'sentiment' task
sample_keys = list(test_dataset[0].keys()) if len(test_dataset) > 0 else []
if not any(k in ('sentiment', 'label', 'label_id') for k in sample_keys):
    local_base = os.path.join('tweet_eval', 'datasets', 'sentiment')
    text_path = os.path.join(local_base, 'test_text.txt')
    label_path = os.path.join(local_base, 'test_labels.txt')
    if os.path.exists(text_path) and os.path.exists(label_path):
        texts = open(text_path, 'r', encoding='utf-8').read().splitlines()
        labels = [int(x) for x in open(label_path, 'r', encoding='utf-8').read().splitlines()]
        test_dataset = Dataset.from_dict({
            'text': texts,
            'sentiment': labels
        })
        print("Loaded local tweet_eval 'sentiment' test split from workspace files.")
    else:
        print("No label fields found and local sentiment files not present.")

# Visualizza le colonne disponibili
try:
    print("Colonne disponibili:", test_dataset.column_names)
except Exception:
    print("Colonne disponibili: ", list(test_dataset[0].keys()) if len(test_dataset) > 0 else [])

# Visualizza un campione
print("\nPrimo campione:")
print(test_dataset[0])

Repo card metadata block was not found. Setting CardData to empty.


Loaded local tweet_eval 'sentiment' test split from workspace files.
Colonne disponibili: ['text', 'sentiment']

Primo campione:
{'text': "@user @user what do these '1/2 naked pics' have to do with anything? They're not even like that. ", 'sentiment': 1}


In [3]:
# Valutazione del modello
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from src.sentiment_model import analyze_sentiment
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Usa i primi 50 campioni etichettati per la valutazione (per velocità)
sample_size = min(50, len(test_dataset))
# Randomly sample a larger pool and then pick labeled samples from it
pool_size = min(len(test_dataset), sample_size * 3)
sample_indices = np.random.choice(len(test_dataset), pool_size, replace=False)

# Mapping delle label
label_mapping = {0: "Negativo", 1: "Neutro", 2: "Positivo"}
reverse_mapping = {"Negativo": 0, "Neutro": 1, "Positivo": 2}

# Valutazione
predictions = []
true_labels = []
confidences = []

for idx in sample_indices:
    sample = test_dataset[int(idx)]
    text = sample['text']
    # Robust label lookup: support several possible field names
    if 'sentiment' in sample:
        raw_label = sample['sentiment']
    elif 'label' in sample:
        raw_label = sample['label']
    elif 'label_id' in sample:
        raw_label = sample['label_id']
    else:
        # Skip samples without labels (some splits may be unlabeled)
        continue
    # Convert numeric label to its string mapping if necessary
    if isinstance(raw_label, int):
        true_label = label_mapping[raw_label]
    else:
        true_label = raw_label
    
    # Effettua la predizione
    result = analyze_sentiment(text)
    pred_label = max(result, key=result.get)
    confidence = result[pred_label]
    
    predictions.append(pred_label)
    true_labels.append(true_label)
    confidences.append(confidence)
    # Stop once we've collected enough labeled samples
    if len(true_labels) >= sample_size:
        break

# If no labeled samples were found, inform the user and skip metric computation
if len(true_labels) == 0:
    print("No labeled samples found in the test dataset; cannot compute metrics.")
else:
    # Calcola le metriche
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average='weighted', zero_division=0)

    print("=" * 60)
    print("VALUTAZIONE DEL MODELLO SUL TEST SET")
    print("=" * 60)
    print(f"\nAccuracy: {accuracy:.4f}")
    print(f"F1 Score (weighted): {f1:.4f}")
    print(f"Confidenza media: {np.mean(confidences):.4f}")
    print(f"\n{classification_report(true_labels, predictions, zero_division=0)}")

VALUTAZIONE DEL MODELLO SUL TEST SET

Accuracy: 0.7800
F1 Score (weighted): 0.7674
Confidenza media: 0.7406

              precision    recall  f1-score   support

    Negativo       0.75      0.86      0.80        14
      Neutro       0.80      0.86      0.83        28
    Positivo       0.75      0.38      0.50         8

    accuracy                           0.78        50
   macro avg       0.77      0.70      0.71        50
weighted avg       0.78      0.78      0.77        50

